In [2]:
# Import libraries
!pip install pandas-datareader
import pandas_datareader.data as web
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as st
from math import exp
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 107 kB 5.8 MB/s eta 0:00:01


In [3]:
# Date range
start = dt.datetime(2019, 1, 28)
end = dt.datetime(2020, 2, 7)

# Stocks from Yahoo finance
# Apple = web.DataReader('AAPL', 'yahoo', start, end)['Adj Close']
# SP_500 = web.DataReader('^GSPC', 'yahoo', start, end)['Adj Close']

Apple = pd.read_csv("apple.csv")
SP_500 = pd.read_csv("SP_500.csv")

#Liabilities
lis = []
for i in range(86):
    lis.append(170.328185)
for i in range(86):
    lis.append(274.610038)
for i in range(88):
    lis.append(362.864864)

liabilities = lis # 260 days liabilities

In [4]:
# Risk-free rate ( One month US Treasury bill from  Fama and Kenneth R. French.)
rf = pd.read_csv('risk_free.csv')
rf = rf.tail(319).head(260)
rf = rf.RF # 260 days risk-free rate

In [5]:
# ### Combining AAPL data and SP_500 data

data = pd.DataFrame({'Equity':Apple['Adj Close'],'sp_500':SP_500['Adj Close']})
data = data.iloc[:-1,:]
data.columns = ['Equity','sp_500']
data['Liabilities'] =lis
data['Risk_free'] = rf.to_list()
data = data[['Equity','Liabilities','Risk_free','sp_500']]
data['Equity'] = data['Equity'] * 153/38.11027

data



,Equity,Liabilities,Risk_free,sp_500
0,152.999026,170.328185,0.010,2643.850098
1,151.413246,170.328185,0.010,2640.000000
2,161.759988,170.328185,0.010,2681.050049
3,162.924888,170.328185,0.010,2704.100098
4,163.003208,170.328185,0.010,2706.530029
...,...,...,...,...
255,307.525482,362.864864,0.006,3225.520020
256,306.680966,362.864864,0.006,3248.919922
257,316.805629,362.864864,0.006,3297.590088
258,319.388949,362.864864,0.006,3334.689941


In [6]:
# Drop sp_500 column
df = data.drop('sp_500', axis = 1)
df['log_risk_free'] = np.log(1+df['Risk_free'])
df = df[['Equity','Liabilities', 'log_risk_free']]
df

,Equity,Liabilities,log_risk_free
0,152.999026,170.328185,0.009950
1,151.413246,170.328185,0.009950
2,161.759988,170.328185,0.009950
3,162.924888,170.328185,0.009950
4,163.003208,170.328185,0.009950
...,...,...,...
255,307.525482,362.864864,0.005982
256,306.680966,362.864864,0.005982
257,316.805629,362.864864,0.005982
258,319.388949,362.864864,0.005982


In [7]:
# initialising all parameters
debt = df.Liabilities
equity = df.Equity
rate = df.log_risk_free
T = 1
book_asset = equity + debt

### Construct functions

In [8]:
def asset_evolution(initial_asset_value, asset_volatility,liability_value, maturity, risk_free_rate,
                    max_time, number_of_runs, resolution):
    
    # Simulates geometric brownian motion asset evolution with the given parameters
    # and returns a numpy matrix of realisations.
    # Simulates up to max_time (useful for visualisation)
    # The resolution parameter sets the number of time steps per unit of time.

    # Convert maturity time into array indices and create the array
    maturity_index = int(maturity * resolution)
    max_time_index = int(max_time * resolution)
    asset_value_matrix = np.zeros((max_time_index, number_of_runs))

    # Initialise the matrix and rescale the volatility parameter
    asset_value_matrix[0,:] = initial_asset_value
    asset_volatility_adjusted = asset_volatility / math.sqrt(resolution)

    # Evolve the asset value
    for i in range(1, max_time_index):
        asset_return = np.random.normal(risk_free_rate / resolution, asset_volatility_adjusted, (1, number_of_runs))
        asset_value_matrix[i, :] = asset_value_matrix[i-1, :] * (1+asset_return)

    return asset_value_matrix


def merton_pd_cf(initial_asset_value, asset_volatility,liability_value, maturity, risk_free_rate):
    # Closed form solution for the Merton model

    def d_minus(a, s, l, t, r):
        return (math.log(a / l) + (r - 0.5 * (s ** 2)) * t) / (s * math.sqrt(t))

    return norm.cdf(-d_minus(initial_asset_value, asset_volatility, liability_value, maturity, risk_free_rate))


def blackcox_pd_cf(initial_asset_value, asset_volatility, liability_value, maturity, risk_free_rate):
    # Closed form solution for the Black and Cox first passage model

    def fp(d, m, t):
        if d < 0:
            d = 0

        return (norm.cdf((d - m * t) / math.sqrt(t)) -
            (math.exp(2 * m * d) * norm.cdf((-d - m * t) / math.sqrt(t))) )

    # k is the rate for discounting the barrier. We set to zero for a fixed face value over time
    k = 0
    m = (1 / asset_volatility) * (risk_free_rate - ((asset_volatility ** 2) / 2) - k)
    d = (1 / asset_volatility) * math.log(liability_value / initial_asset_value)
    return 1 - fp(-d, -m, maturity)

def bc(a, s, l, t, r, runs, resolution):

    # We want to simulate to 20% beyond the maturity time
    max_time = t * 1.2

    # Create vector for the x-axis
    x = np.linspace(0, max_time, int(resolution*max_time))

    # Do the simulation
    asset_value_matrix  = asset_evolution(a, s, l, t, r, max_time, runs, resolution)

    # Create a vector for the liability
    vec1 = np.ones(  int(resolution*t) ) * l
    vec2 = np.zeros( int(resolution*max_time)-int(resolution*t))
    liability_bar = np.concatenate( (vec1, vec2) )

    # Plot the 'barrier' in red and the asset value realisations in slightly transparent black.
    plt.bar(x, liability_bar, width=(1/resolution), facecolor='r', edgecolor='r', alpha=1)
    plt.plot(x, asset_value_matrix, 'k', alpha=0.2, linewidth=1)
    plt.axis([0, max_time, 1e6, 0.12e7])

    # Calculate the proportion of occurances where the asset value is ever below liability value
    tf = asset_value_matrix[:, :] - l
    # For each row, check to see if it ever went negative
    count = 0
    for i in range(1, runs):
        if sum(tf[:,i] < 0) > 0:
            count = count + 1

    global credit_classification
    credit_classification = "Aaa"

    pd = count / runs

    global estimatedBCPD
    global actualBCPD
    estimatedBCPD.value = round(pd, 4)
    actualBCPD.value    = round(blackcox_pd_cf(a, s, l, t, r), 4)

In [9]:
asset = book_asset.tolist()
SSE = 1                  # Initializing random sum squared difference value
asset_series = []        # Save the list of asset into the asset series list.
SSE_series = []          # Save error value from each iteration.
asset_vol_series = []    # Save the value of volatility from each iteration

number_iter = 10

for i in range(number_iter):
    asset_series.append(asset)

    if SSE <= 1e-10:
        asset_vol_series.append(asset_vol)
        break
    else:

        log_returns = [np.log(asset[i+1]/asset[i]) for i in range(len(asset)-1)]  # Compute asset returns
        asset_vol = np.std(log_returns)*260**0.5    # Compute asset volatility

        asset_vol_series.append(asset_vol)

        d1 = (np.log(asset/debt)+(rate + (asset_vol**2)*0.5)*T)/asset_vol*np.sqrt(T)
        d2 = d1 - asset_vol*np.sqrt(T)
        df['d1'] = d1
        df['d2'] = d2     # Save d1 and d2 dataframe containing (equity, debt,log_risk_free)

        current_asset = []     # List to hold all the daily asset value for the current iteration

        for record in df.to_records(index=False): # record = (equity, debt,log_risk_free, d1,d2)

            #Using Black_Scholes_Merton formula to compute currrent asset value in the market
            equity_,debt_,rate_,d1,d2 = record
            daily_asset = (equity_ + debt_*exp(-rate_*T)*st.norm.cdf(d2))/st.norm.cdf(d1)

            current_asset.append(daily_asset)


        SSE = np.sum([(i-j)**2  for i,j in zip(asset,current_asset)]) # Compute the sum squared difference
        SSE_series.append(SSE)

        asset = current_asset # Updating for next iteration

df_aseet_iter = pd.DataFrame(asset_series) # Put all asset iterations into one Dataframe

asset_iteration_df = df_aseet_iter.T
asset_iteration_df

,0,1,2,3,4,5,6,7
0,323.327211,321.070710,321.150246,321.144504,321.144948,321.144914,321.144917,321.144916
1,321.741431,319.461525,319.543523,319.537597,319.538056,319.538020,319.538023,319.538023
2,332.088173,329.945151,330.012265,330.007446,330.007818,330.007790,330.007792,330.007792
3,333.253073,331.123312,331.188916,331.184209,331.184573,331.184545,331.184547,331.184547
4,333.331393,331.202509,331.268013,331.263313,331.263676,331.263648,331.263650,331.263650
...,...,...,...,...,...,...,...,...
255,670.390346,666.683789,666.887949,666.873213,666.874351,666.874264,666.874270,666.874270
256,669.545830,665.823700,666.029399,666.014548,666.015695,666.015607,666.015614,666.015613
257,679.670493,676.125040,676.312961,676.299432,676.300478,676.300397,676.300404,676.300403
258,682.253813,678.750099,678.933714,678.920505,678.921526,678.921447,678.921453,678.921453


In [10]:
credit_classification = ['Aaa', 'Aaa']

In [11]:
# This dataframe contained risk free rate, S&P500, asset, last iterated asset
df_drift = data[['Risk_free','sp_500']]
df_drift['book_asset'] = asset_iteration_df[0].tolist()
df_drift['asset_iter'] = asset_iteration_df[7].tolist()


risk_free_rate = df_drift['Risk_free'].tolist()
rf = [i for i in risk_free_rate]
rf.remove(0.01)  # Removing 0.01 from the rf in order to match the same size as the asset returns.



In [12]:
df_drift

,Risk_free,sp_500,book_asset,asset_iter
0,0.010,2643.850098,323.327211,321.144916
1,0.010,2640.000000,321.741431,319.538023
2,0.010,2681.050049,332.088173,330.007792
3,0.010,2704.100098,333.253073,331.184547
4,0.010,2706.530029,333.331393,331.263650
...,...,...,...,...
255,0.006,3225.520020,670.390346,666.874270
256,0.006,3248.919922,669.545830,666.015613
257,0.006,3297.590088,679.670493,676.300403
258,0.006,3334.689941,682.253813,678.921453


In [13]:
# S&P500
sp_500 = df_drift['sp_500'].tolist()
sp_500_ret = [(sp_500[i+1] - sp_500[i])/sp_500[i] for i in range(len(sp_500)-1)] # Compute daily returns for S&P500
sp_r1 = [i for i in sp_500_ret]
sp_excess_r1 = [i-j for i,j in zip(sp_r1,rf)]   # Compute excess return for S&P500
V_sp_excess_r1 = [i for i in sp_excess_r1]
V_sp_excess_r1.insert(0,np.nan) # Fill in nan value in order to match the size of assets in the dataframe below
sp_r1.insert(0,np.nan)    # Fill in nan value in order to match the size of S&p500 in the dataframe below

# Book asset
book_asset = df_drift['book_asset'].tolist()
book_asset_ret = [(book_asset[i+1] - book_asset[i])/book_asset[i] for i in range(len(book_asset)-1)]
ba_r2 = [i for i in book_asset_ret]
excess_ba_r2 = [i-j for i,j in zip(ba_r2,rf)]
V_excess_ba_r2 = [i for i in excess_ba_r2]
V_excess_ba_r2.insert(0,np.nan)
ba_r2.insert(0,np.nan)

# Asset Iterate
asset_iter = df_drift['asset_iter'].tolist()
asset_iter_ret = [(asset_iter[i+1] - asset_iter[i])/asset_iter[i] for i in range(len(asset_iter)-1)]
ait_r2 = [i for i in asset_iter_ret]
excess_ait_r2 = [i-j for i,j in zip(ait_r2,rf)]
V_excess_ait_r2 = [i for i in excess_ait_r2]
V_excess_ait_r2.insert(0,np.nan)
ait_r2.insert(0,np.nan)

# Dataframe
df_drift['sp_500_returns'] = sp_r1
df_drift['book_asset_ret'] = ba_r2
df_drift['asset_iter_ret'] = ait_r2
df_drift['V_sp_excess_r1'] = V_sp_excess_r1
df_drift['V_excess_ba_r2'] = V_excess_ba_r2
df_drift['V_excess_ait_r2'] = V_excess_ait_r2
df_drift

,Risk_free,sp_500,book_asset,asset_iter,sp_500_returns,book_asset_ret,asset_iter_ret,V_sp_excess_r1,V_excess_ba_r2,V_excess_ait_r2
0,0.010,2643.850098,323.327211,321.144916,NaN,NaN,NaN,NaN,NaN,NaN
1,0.010,2640.000000,321.741431,319.538023,-0.001456,-0.004905,-0.005004,-0.011456,-0.014905,-0.015004
2,0.010,2681.050049,332.088173,330.007792,0.015549,0.032159,0.032765,0.005549,0.022159,0.022765
3,0.010,2704.100098,333.253073,331.184547,0.008597,0.003508,0.003566,-0.001403,-0.006492,-0.006434
4,0.010,2706.530029,333.331393,331.263650,0.000899,0.000235,0.000239,-0.009101,-0.009765,-0.009761
...,...,...,...,...,...,...,...,...,...,...
255,0.006,3225.520020,670.390346,666.874270,-0.017706,-0.020840,-0.021260,-0.023706,-0.026840,-0.027260
256,0.006,3248.919922,669.545830,666.015613,0.007255,-0.001260,-0.001288,0.001255,-0.007260,-0.007288
257,0.006,3297.590088,679.670493,676.300403,0.014980,0.015122,0.015442,0.008980,0.009122,0.009442
258,0.006,3334.689941,682.253813,678.921453,0.011251,0.003801,0.003876,0.005251,-0.002199,-0.002124


In [14]:
excess_df = pd.DataFrame([sp_excess_r1,excess_ba_r2,excess_ait_r2]).T
excess_df.columns = ['excess_sp_500', 'excess_ba', 'excess_ait']
excess_df

,excess_sp_500,excess_ba,excess_ait
0,-0.011456,-0.014905,-0.015004
1,0.005549,0.022159,0.022765
2,-0.001403,-0.006492,-0.006434
3,-0.009101,-0.009765,-0.009761
4,-0.003224,0.003890,0.004109
...,...,...,...
254,-0.023706,-0.026840,-0.027260
255,0.001255,-0.007260,-0.007288
256,0.008980,0.009122,0.009442
257,0.005251,-0.002199,-0.002124


In [15]:
# Calculate Beta from book asset and market index (S&P500)
cov_sp_ba = excess_df[['excess_sp_500','excess_ba']].cov()
Beta_sp_ba = cov_sp_ba.loc['excess_sp_500','excess_ba']/excess_df[['excess_sp_500']].var()
Beta_sp_ba = Beta_sp_ba.values[0]

# Calculate Beta from asset iterate and market index (S&P500)
cov_sp_ait = excess_df[['excess_sp_500','excess_ait']].cov()
Beta_sp_ait = cov_sp_ait.loc['excess_sp_500','excess_ait']/excess_df[['excess_sp_500']].var()
Beta_sp_ait = Beta_sp_ait.values[0]
print((Beta_sp_ba,Beta_sp_ait))

(0.17997462528076247, 0.21355148754029707)


In [16]:
# We convert the average daily return to average anualised daily returns:
E_Rm = (np.mean(sp_500_ret) + 1)**259 - 1

# average annualised market risk premium is assumed :
market_premium = E_Rm - rf[-1]
market_premium

0.2686491063105685

In [17]:
#Expected return on book asset
expected_return_1 =  rf[-1] + (Beta_sp_ba * market_premium)
#Expected return on asset iterate
expected_return_2 =  rf[-1] + (Beta_sp_ait*market_premium)

(expected_return_1,expected_return_2)

(0.05435002224025628, 0.06337041627899331)

In [18]:
# drift rate of book asset
drift_1 = np.log(1+expected_return_1 )
# drift rate of asset iterate
drift_2 = np.log(1+expected_return_2 )

(drift_1,drift_2)

(0.05292448440169452, 0.0614435017641187)

In [19]:
capm_variable = {'At maturity 2020-02-06':['book asset', 'asset iterate'],
                 'Beta':[Beta_sp_ba,Beta_sp_ait],
                 'Expected return':[expected_return_1,expected_return_2],
                 'Drift rate': [drift_1,drift_2]}
pd.DataFrame(capm_variable)

,At maturity 2020-02-06,Beta,Expected return,Drift rate
0,book asset,0.179975,0.05435,0.052924
1,asset iterate,0.213551,0.06337,0.061444


In [20]:
debt_last = debt[259]   # Debt at maturity

In [21]:
# Book asset
asset_1 = asset_iteration_df[0][259]  #Book asset at maturity
asset_1_vol = asset_vol_series[0]    #Book_asset's volatility at maturity
mu_1 = drift_1    #Drift rate at maturity

# Probability of default for Book asset
PD = st.norm.cdf(-((np.log(asset_1/debt_last) + (mu_1 - (asset_1_vol**2)*0.5)*T)/asset_1_vol*np.sqrt(T)))

# Asset iterate
asset_iter = asset_iteration_df[7][259]   # Asset_iterate at maturity
asset_iter_vol = asset_vol_series[6]    # Asset_iterate's volatility at maturity
mu_2 = drift_2   # Drift rate at maturity

#probability of default for Asset iterate
PD_iter = st.norm.cdf(-((np.log(asset_iter/debt_last) + (mu_1 - (asset_iter_vol**2)*0.5)*T)/asset_iter_vol*np.sqrt(T)))

print(str(round(PD*100,2))+'%', str(round(PD_iter*100,2))+'%')

2.19% 2.01%


In [22]:
PD_variables = {'At maturity 2020-02-06' :['Debt ($L_{T}$)','Asset ($A_{T}$)','Volatility ($\sigma_{A}$)',
                                           'Drift rate ($\mu_{A}$)',' ','Probability of default ($PD$)'],
                'Book asset':[debt_last, asset_1, str(round(asset_1_vol,2)) +'%', str(round(mu_1*100,2)) + '%', ' ', str(round(PD*100,2)) + '%'],
                'Asset iterate':[debt_last, asset_iter, str(round(asset_iter_vol,2)) +'%', str(round(mu_2*100,2)) +'%',' ', str(round(PD_iter*100,2)) + '%']}

pd.DataFrame(PD_variables)

,At maturity 2020-02-06,Book asset,Asset iterate
0,Debt ($L_{T}$),362.865,362.865
1,Asset ($A_{T}$),685.99,682.71
2,Volatility ($\sigma_{A}$),0.32%,0.31%
3,Drift rate ($\mu_{A}$),5.29%,6.14%
4,,,
5,Probability of default ($PD$),2.19%,2.01%


In [23]:
transition_df = pd.read_csv('trasition.csv',index_col=0)

In [24]:
print("transition metric")
print(transition_df)

transition metric
      Aaa      Aa       A     Baa      Ba       B      D
0  91.027   7.003   2.000   0.299   0.151   0.007    0.0
1   6.998  85.823  10.865   0.999   0.902   0.047    0.0
2   1.003   5.997  80.251   3.798   3.701   0.217    0.0
3   0.650   0.704   6.159  90.624   7.002   0.405    0.0
4   0.238   0.266   0.397   3.680  72.855   8.898    0.0
5   0.059   0.147   0.238   0.400  12.889  78.849    0.0
6   0.025   0.060   0.090   0.200   2.500  11.576  100.0


In [25]:
print('Classification:')
print(credit_classification[0])

Classification:
Aaa


In [26]:
print("Credit migration:")
for item in transition_df.dot(transition_df.T).iloc[0]:
    item = item/transition_df.dot(transition_df.T).iloc[0].sum() * 100
    print(item)

Credit migration:
82.98654547164368
12.540832623900188
2.9406295760232077
1.0406557571909394
0.3630811450776289
0.09447805195312295
0.03377737421123545
